# Classification

#### Data Sourced from:
https://www.kaggle.com/mlg-ulb/creditcardfraud

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import time

In [125]:
# sklearn packages
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

In [25]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [5]:
raw_data = pd.read_csv(r'C:\Users\pmven\Google Drive\1. myDocs\Projects\CC Fraud\creditcardfraud\creditcard.csv')

In [6]:
raw_data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

Data contains 31 columns in total <br>
V1 - V28 are features <br>
Amount, and class are the fraud amounts and flags. 
What is time?

In [27]:
round(raw_data.head(5), 2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000,-1.360,-0.070,2.540,1.380,-0.340,0.460,0.240,0.100,0.360,...,-0.020,0.280,-0.110,0.070,0.130,-0.190,0.130,-0.020,149.620,0
1,0.000,1.190,0.270,0.170,0.450,0.060,-0.080,-0.080,0.090,-0.260,...,-0.230,-0.640,0.100,-0.340,0.170,0.130,-0.010,0.010,2.690,0
2,1.000,-1.360,-1.340,1.770,0.380,-0.500,1.800,0.790,0.250,-1.510,...,0.250,0.770,0.910,-0.690,-0.330,-0.140,-0.060,-0.060,378.660,0
3,1.000,-0.970,-0.190,1.790,-0.860,-0.010,1.250,0.240,0.380,-1.390,...,-0.110,0.010,-0.190,-1.180,0.650,-0.220,0.060,0.060,123.500,0
4,2.000,-1.160,0.880,1.550,0.400,-0.410,0.100,0.590,-0.270,0.820,...,-0.010,0.800,-0.140,0.140,-0.210,0.500,0.220,0.220,69.990,0


In [10]:
# Target Variable
raw_data['Class'].unique()

array([0, 1], dtype=int64)

In [21]:
# how is the variable distributed?
raw_data[['Class']].groupby('Class').agg('size')

Class
0    284315
1       492
dtype: int64

The data seems to be quite imbalced. We need to make stratified sampling in order to balance out the dataset

In [28]:
# How are the variables distributed?
round(raw_data.describe(),2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,...,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000,284807.000
mean,94813.860,0.000,0.000,-0.000,0.000,0.000,0.000,-0.000,0.000,-0.000,...,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,88.350,0.000
std,47488.150,1.960,1.650,1.520,1.420,1.380,1.330,1.240,1.190,1.100,...,0.730,0.730,0.620,0.610,0.520,0.480,0.400,0.330,250.120,0.040
min,0.000,-56.410,-72.720,-48.330,-5.680,-113.740,-26.160,-43.560,-73.220,-13.430,...,-34.830,-10.930,-44.810,-2.840,-10.300,-2.600,-22.570,-15.430,0.000,0.000
25%,54201.500,-0.920,-0.600,-0.890,-0.850,-0.690,-0.770,-0.550,-0.210,-0.640,...,-0.230,-0.540,-0.160,-0.350,-0.320,-0.330,-0.070,-0.050,5.600,0.000
50%,84692.000,0.020,0.070,0.180,-0.020,-0.050,-0.270,0.040,0.020,-0.050,...,-0.030,0.010,-0.010,0.040,0.020,-0.050,0.000,0.010,22.000,0.000
75%,139320.500,1.320,0.800,1.030,0.740,0.610,0.400,0.570,0.330,0.600,...,0.190,0.530,0.150,0.440,0.350,0.240,0.090,0.080,77.160,0.000
max,172792.000,2.450,22.060,9.380,16.880,34.800,73.300,120.590,20.010,15.590,...,27.200,10.500,22.530,4.580,7.520,3.520,31.610,33.850,25691.160,1.000


In [37]:
# Still dont understand what time is.
# What are the unique values?
print("Unique Value Count:",raw_data['Time'].unique().size)
print('unique Values:', raw_data['Time'].unique())

Unique Value Count: 124592
unique Values: [0.00000e+00 1.00000e+00 2.00000e+00 ... 1.72787e+05 1.72788e+05
 1.72792e+05]


https://www.kaggle.com/mlg-ulb/creditcardfraud/discussion/30627<br>
basically it is the time elapsed in seconds between first transaction and all subsequent transactions
<br>Keeping it aside for now.

In [94]:
raw_data.iloc[:,1:29].columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28'],
      dtype='object')

In [68]:
X = raw_data.iloc[:,1:30]
y = raw_data.iloc[:,30:]
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1, stratify = y)

In [69]:
# Making sure stratified sampling has taken place
print("Original Data \n", y.groupby('Class').agg('size'), "\n")
print("Train Data \n", y_train.groupby('Class').agg('size'), "\n")
print("Test Data \n", y_test.groupby('Class').agg('size'), "\n")

Original Data 
 Class
0    284315
1       492
dtype: int64 

Train Data 
 Class
0    199020
1       344
dtype: int64 

Test Data 
 Class
0    85295
1      148
dtype: int64 



#### The data seem to have split in 70 - 30 fashion for the class variable

## Logistic Regression

In [70]:
from sklearn.linear_model import LogisticRegression

In [80]:
logit = LogisticRegression(random_state=123)
logit.fit(x_train, y_train)
y_pred = logit.predict(x_test)

In [82]:
accuracy_score(y_pred, y_test)* 100

99.93328885923948

We have pretty Good accuracy. but how does the confusion matrix look?

In [106]:
matrix = confusion_matrix(y_pred, y_test)
print("   0        1")
print(matrix)

print("\nRecall:", round(recall_score(y_test, y_pred, pos_label=1)*100,2), "%")
print("Precision: ", round(precision_score(y_test, y_pred, pos_label = 1)*100, 2), "%")

   0        1
[[85284    46]
 [   11   102]]

Recall: 68.92 %
Precision:  90.27 %


The recall score for this matrix looks pretty poor
The algorithm fails to identify 46 of the 102 fraudulant transactions

In [93]:
# What if we remove the Amount variable?

In [95]:
X = raw_data.iloc[:,1:29]
y = raw_data.iloc[:,30:]
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1, stratify = y)

In [96]:
logit = LogisticRegression(random_state=123)
logit.fit(x_train, y_train)
y_pred = logit.predict(x_test)

In [97]:
accuracy_score(y_pred, y_test)* 100

99.93328885923948

We have pretty Good accuracy. but how does the confusion matrix look?

In [115]:
matrix = confusion_matrix(y_pred, y_test)
print("   0        1")
print(matrix)

print("\nRecall:", round(recall_score(y_test, y_pred, pos_label=1)*100,2), "%")
print("Precision: ", round(precision_score(y_test, y_pred, pos_label = 1)*100, 2), "%")

   0        1
[[85284    46]
 [   11   102]]

Recall: 68.92 %
Precision:  90.27 %


**Makes no Difference**

### Performing Cross Validation to get actual model performance

In [118]:
scores = cross_val_score(logit, x_train, y_train, cv = 10)
print(round(np.mean(scores), 2))

1.0


The model scores are consistent with what is observed previously. 99% accuracy. But the model needs to be tuned to get the max recall, which is a low 68% right now.

### Hyperparameter Tuning to get the maximum recall for the model

In [136]:
# Creating the hyperparameter space
penalty = ['l1', 'l2']
C = np.arange(0.1,4,0.5)
grid_dict = dict(penalty=penalty, C=C)

In [137]:
grid = GridSearchCV(logit, grid_dict, cv = 3)
best_logit = grid.fit(x_train, y_train)

In [143]:
best_logit.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=123, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([0.1, 0.6, 1.1, 1.6, 2.1, 2.6, 3.1, 3.6]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)>

#### The model performs best when C = 1 and l2 penalty

In [144]:
X = raw_data.iloc[:,1:29]
y = raw_data.iloc[:,30:]
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1, stratify = y)

In [149]:
logit = LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=123, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False)
logit.fit(x_train, y_train)
y_pred = logit.predict(x_test)

In [150]:
matrix = confusion_matrix(y_pred, y_test)
print("   0        1")
print(matrix)

print("\nRecall:", round(recall_score(y_test, y_pred, pos_label=1)*100,2), "%")
print("Precision: ", round(precision_score(y_test, y_pred, pos_label = 1)*100, 2), "%")

   0        1
[[85284    46]
 [   11   102]]

Recall: 68.92 %
Precision:  90.27 %


### Logistic Regression Does not perform very well